In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

tqdm.pandas()

In [3]:
structure = pd.read_csv("kaggle/structure.csv")
targets = pd.read_csv("kaggle/targets.csv")
df = pd.read_csv("kaggle/user_activity.csv")

In [4]:
costed_steps = structure[structure.step_cost > 0]

In [9]:
df[(df.step_id.isin(costed_steps.step_id)) & (df.user_id == 16953)]

,user_id,action,step_id,time,step_type,step_cost
154347,16953,discovered,4289,1410894805,choice,1
154348,16953,viewed,4289,1410894805,choice,1
154355,16953,started_attempt,4289,1410894809,choice,1
361732,16953,discovered,2918,1411490384,code,2
361733,16953,viewed,2918,1411490384,code,2
361737,16953,discovered,2919,1411490389,code,2
361738,16953,viewed,2919,1411490389,code,2
361740,16953,discovered,4969,1411490391,code,2
361741,16953,viewed,4969,1411490391,code,2
361760,16953,discovered,7422,1411490400,code,2


In [123]:
def f(x):
    passed = x[x.action == "passed"]
    not_passed = x[~(x.step_id.isin(passed.step_id))]
    
    #lol-comprehension
    avg_attempts = np.mean([t.time.values.size for t in [x[(x.action == "started_attempt") & (x.step_id == id_)] for id_ in x.step_id.unique()]])
    
    not_passed = not_passed.drop_duplicates("step_id")
#     print(not_passed)

    days = 10*3600*24
    border = passed.time.max() - days
    passed_steps = len([1 for t in passed.time if t >= border])
    skipped_steps = not_passed.time.values.size
    
    passed_balls = passed.step_cost.sum()
    viewed_balls = not_passed[not_passed.action == "viewed"].step_cost.sum()
#     if not viewed_balls:
#         q = 0
#     else:
#         q = passed_balls / viewed_balls
    
    res = [passed_steps, skipped_steps, avg_attempts, passed_balls, viewed_balls]
    return res

# TODO: Number of skipped steps and substraction of passed and skipped

In [124]:
x = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").get_group(16953)

In [125]:
f(x)

[10, 4, 1.8571428571428572, 14, 2]

In [126]:
X_t = df[df.step_id.isin(costed_steps.step_id)].groupby("user_id").progress_apply(f)
Y_t = targets[targets.user_id.isin(X_t.index)]





  0%|          | 0/9426 [00:00<?, ?it/s]



  0%|          | 2/9426 [00:00<12:54, 12.17it/s]



  0%|          | 3/9426 [00:00<15:01, 10.45it/s]



  0%|          | 7/9426 [00:00<11:57, 13.12it/s]



  0%|          | 11/9426 [00:00<09:52, 15.89it/s]



  0%|          | 14/9426 [00:00<08:31, 18.42it/s]



  0%|          | 18/9426 [00:00<07:34, 20.69it/s]



  0%|          | 21/9426 [00:00<07:50, 19.99it/s]



  0%|          | 24/9426 [00:01<07:13, 21.68it/s]



  0%|          | 27/9426 [00:01<07:42, 20.31it/s]



  0%|          | 32/9426 [00:01<06:23, 24.50it/s]



  0%|          | 37/9426 [00:01<05:49, 26.89it/s]



  0%|          | 42/9426 [00:01<05:06, 30.64it/s]



  1%|          | 49/9426 [00:01<04:16, 36.57it/s]



  1%|          | 54/9426 [00:01<04:59, 31.27it/s]



  1%|          | 59/9426 [00:02<04:50, 32.29it/s]



  1%|          | 63/9426 [00:02<05:03, 30.83it/s]



  1%|          | 67/9426 [00:02<05:16, 29.56it/s]



  1%|          | 71/9426 [00:02<05:17, 29.45it/s]



 

In [127]:
X_t.values, Y_t.passed.values

(array([[0, 68, 0.0, 0, 51], [9, 0, 2.3333333333333335, 9, 0],
        [9, 0, 2.6666666666666665, 9, 0], ...,
        [2, 1, 3.3333333333333335, 2, 1], [0, 2, 1.0, 0, 2],
        [3, 0, 6.0, 3, 0]], dtype=object), array([0, 0, 0, ..., 0, 0, 0]))

In [128]:
X = np.array([np.array(x) for x in X_t.values]).reshape(-1, 5)
Y = Y_t.passed.ravel()

In [129]:
X

array([[  0.        ,  68.        ,   0.        ,   0.        ,  51.        ],
       [  9.        ,   0.        ,   2.33333333,   9.        ,   0.        ],
       [  9.        ,   0.        ,   2.66666667,   9.        ,   0.        ],
       ..., 
       [  2.        ,   1.        ,   3.33333333,   2.        ,   1.        ],
       [  0.        ,   2.        ,   1.        ,   0.        ,   2.        ],
       [  3.        ,   0.        ,   6.        ,   3.        ,   0.        ]])

In [130]:
classifier = SGDClassifier(loss="squared_hinge")
classifier.coef_ = [2.3696734 , -3.32163541, -0.57359424, -7.78178374]
classifier.intercept_ = [-24.63634201]

In [131]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [132]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [133]:
from sklearn.metrics import f1_score
f1_score(Y_test, Y_pred)

0.38683127572016462

In [134]:
best_params = [None, None]
scores = []
best_score = 0
for _ in tqdm_notebook(range(1000)):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
    if Y_pred.sum() > 0:
        score = f1_score(Y_test, Y_pred)
        if score > best_score:
            print(Y_pred.sum())
            best_score = max(best_score, score)
#             best_params = [classifier.coef_, classifier.intercept_] 
        scores.append(score)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

75


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

79


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

...

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [135]:
np.median(scores)
np.mean(scores)

0.29059829059829057

0.28975509752468825

In [136]:
best_params

[None, None]

In [137]:
def prepare(df, num_features):
    X = df.groupby("user_id").progress_apply(f)
    return X.index, np.array([np.array(x) for x in X.values]).reshape(-1, num_features)

In [139]:
user_ids, X_val = prepare(pd.read_csv("kaggle/user_activity_test.csv"), 5)


100%|██████████| 4256/4256 [04:26<00:00, 15.96it/s]

In [140]:
X_train[5], X_val[5]

(array([ 3.        ,  0.        ,  2.33333333,  3.        ,  0.        ]),
 array([ 30.        ,   0.        ,   0.16666667,   3.        ,   0.        ]))

In [141]:
predicted = classifier.predict(X_val)

In [142]:
res = pd.DataFrame({"user_id": user_ids, "passed": predicted}).set_index("user_id")

In [143]:
res.passed.value_counts()

0    4140
1     115
Name: passed, dtype: int64

In [144]:
res.to_csv("result.csv")